## Заполнение онтологии из файла

In [6]:
from owlready2 import *
import types
import csv
import codecs

#Функция для заполнения онтологии с готовой структурой классов инстансами из справочников
def populate_ontology_indidviduals(onto_path, txt_path, save=True, encoding='cp1251'):
    onto = get_ontology(onto_path).load()
    with codecs.open(txt_path, 'r', encoding) as fp:
        data = fp.readlines()
        
    try:
        if 'environment' in txt_path:
            Class = onto.Environment
        elif 'food' in txt_path:
            Class = onto.Food
        elif 'medication' in txt_path:
            Class = onto.Medication
        elif 'reaction' in txt_path:
            Class = onto.Manifestation
    except NoCategoryInOntologyError:
        Class = onto.Other
        
    with onto:
        for row in data[1:]:
            id_, parent, snomed_name, snomed_code, snomed_partial_code, is_direct = row.split('\t')
                    
            if id_:
                individual = Class(id_)
                          
            if snomed_name:
                individual.snomedName = snomed_name
        
            if snomed_code:
                individual.snomedCode = snomed_code
        
            if snomed_partial_code:
                individual.snomedPartialCode = snomed_partial_code
            
            if is_direct==1:
                isDirect = True
            else:
                isDirect = False
            individual.isDirect = isDirect
            
            if save:
                onto.save(onto_path)
    return onto

In [12]:
onto_path = "Allergy_ontology.owl"
txt_path = 'environment.txt'
onto = populate_ontology_indidviduals(onto_path, txt_path, save=False)
txt_path = 'medication.txt'
onto = populate_ontology_indidviduals(onto_path, txt_path, save=False)
txt_path = 'food.txt'
onto = populate_ontology_indidviduals(onto_path, txt_path, save=False)
txt_path = 'reactions.txt'
onto = populate_ontology_indidviduals(onto_path, txt_path, save=True)

In [8]:
print(onto.Яблоко.is_a)
print(onto.крапивница.is_a)
print(onto.Голубь.is_a)
print(onto.пенициллин.is_a)

[Allergy_ontology.Food]
[Allergy_ontology.Manifestation]
[Allergy_ontology.Environment]
[Allergy_ontology.Medication]


In [9]:
print(onto.Яблоко.name, onto.Яблоко.snomedName, onto.Яблоко.snomedCode)

Яблоко Allergy to apple (finding) 860604008


In [10]:
print('Количество терминов:',len(list(onto.individuals())))

Количество терминов: 2915


In [11]:
#Как создать правильный класс
#Вариант 1
#with onto:
#    class бассейн(onto.Environment): pass

#Вариант 2 - динамический
#with onto:
#    for row in data:
#        id_, parent, snomed_name, snomed_code, snomed_partial_code, is_direct = row.split('\t')
#       
#        if parent: 
#            parent = 'Environment'
#        else:
#            parent = [Thing]
#        Class = types.new_class(id_, (onto[parent],))